In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import random
from tqdm import tqdm

s = requests.session()       
s.keep_alive = False

def get_ret(url, headers, times=20):
    for i in range(times):
        try:
            return requests.get(url, headers=headers)
        except:
            pass
        time.sleep(random.random())
    print (f'url "{url}" not get')
    return None

def find_usr(url='https://www.metacritic.com/game/playstation-4/the-last-of-us-part-ii/user-reviews?sort-by=date&num_items=100',n_iter=10):
    
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
    
    score_list = []
    usr_list = []
    commit_list = []
    for i in range(n_iter):
        url_now = url + f'&page={i}'
        
        ret = get_ret(url_now, headers)
        if ret == None:
            print ('page Error:', i)
            continue
        print (f'page {i} get ret')
        soup = BeautifulSoup(ret.text, 'html.parser')
    
        scores = soup.find_all('div', class_="review_grade")[:-3]
        score_list += [int(score.find('div').string) for score in scores]
        
        commits = soup.find_all('div', class_="review_body")[:-3]
        commit_list += [commit.text for commit in commits]
        commit_length_list = [len(commit.split()) for commit in commit_list]
        
        users = soup.find_all('div', class_='review_critic')[:-3]

        for user in tqdm(users):
            
            usr_url = 'https://www.metacritic.com' + user.a['href']
            name = user.a.text
            rates = 0
            reviews = 0
            flag = False
            
            for i in range(10):
                try:
                    usr_ret = requests.get(usr_url, headers=headers)
                    usr_soup = BeautifulSoup(usr_ret.text, 'html.parser')
                    rates = int(usr_soup.find('span', class_='total_summary_reviews').span.text)
                    reviews = int(usr_soup.find('span', class_='total_summary_ratings mr20').span.text)
                    flag = True
                    break
                except:
                    pass
            if flag == False:
                print ('user Error:', name)
                
            usr_list.append((rates, reviews, name, flag))
            time.sleep(random.random() * 0.6)
            
        length_count = [[] for _ in range(11)]             
        rates_count = [[] for _ in range(11)]
        reviews_count = [[] for _ in range(11)]
        
        for i in range(len(score_list)):
            score = score_list[i]
            rates_count[score].append(usr_list[i][0])
            reviews_count[score].append(usr_list[i][1])
            length_count[score].append(commit_length_list[i])
          
        tmp = np.array(score_list)          
        df = [(sum(tmp==i), str(round(sum(tmp==i) * 100./len(tmp), 1))+'%', round(np.mean(length_count[i]), 1), 
               round(np.mean(rates_count[i]), 1), round(np.mean(reviews_count[i]), 1)) for i in range(11)]
        df = pd.DataFrame(df)
        df.index.name = 'score'
        df.columns = ['count', 'percentage', 'average commit length', 'average usr rates', 'average usr reviews']
        yield df, score_list, commit_list, usr_list
        time.sleep(random.random() * 3)

In [2]:
# headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
# url='https://www.metacritic.com/user/arklles'
# ret = requests.get(url, headers=headers)
# soup = BeautifulSoup(ret.text, 'html.parser')
# int(soup.find('span', class_='total_summary_reviews').span.text)

In [3]:
output_path = 'output/death stranding/'
results = []
for i, result in enumerate(find_usr(url='https://www.metacritic.com/game/playstation-4/death-stranding/user-reviews?sort-by=date&num_items=100', 
                               n_iter=10)):
    df = result[0]
    df.to_csv(output_path+f'death-stranding_df_latest_{i}_pages.csv')
    print (f'death-stranding_df_latest_{i}_pages.csv saved')
    results.append(result)

page 0 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:35<00:00,  2.75s/it]
C:\Users\Administrator\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Administrator\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


death-stranding_df_latest_0_pages.csv saved
page 1 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:43<00:00,  2.83s/it]


death-stranding_df_latest_1_pages.csv saved
page 2 get ret


...

death-stranding_df_latest_7_pages.csv saved
page 8 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:53<00:00,  2.93s/it]


death-stranding_df_latest_8_pages.csv saved
page 9 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:55<00:00,  2.96s/it]


death-stranding_df_latest_9_pages.csv saved


In [4]:
output_path = 'output/red dead redemption 2/'
results2 = []
for i, result in enumerate(find_usr(url='https://www.metacritic.com/game/playstation-4/red-dead-redemption-2/user-reviews?sort-by=date&num_items=100', 
                               n_iter=30)):
    df = result[0]
    df.to_csv(output_path+f'red-dead-redemption-2_df_latest_{i}_pages.csv')
    print (f'red-dead-redemption-2_df_latest_{i}_pages.csv saved')
    results2.append(result)

page 0 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:48<00:00,  2.88s/it]


red-dead-redemption-2_df_latest_0_pages.csv saved
page 1 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [09:48<00:00,  5.89s/it]


red-dead-redemption-2_df_latest_1_pages.csv saved
page 2 get ret


...

red-dead-redemption-2_df_latest_27_pages.csv saved
page 28 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:19<00:00,  3.19s/it]


red-dead-redemption-2_df_latest_28_pages.csv saved
page 29 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:17<00:00,  3.18s/it]


red-dead-redemption-2_df_latest_29_pages.csv saved


In [2]:
output_path = 'output/tlou2/'
results3 = []
for i, result in enumerate(find_usr(n_iter=120)):
    df = result[0]
    df.to_csv(output_path+f'tlou2_df_latest_{i}_pages.csv')
    print (f'tlou2_df_latest_{i}_pages.csv saved')
    results3.append(result)

page 0 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:44<00:00,  2.84s/it]


tlou2_df_latest_0_pages.csv saved
page 1 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:46<00:00,  2.86s/it]


tlou2_df_latest_1_pages.csv saved
page 2 get ret


...

tlou2_df_latest_117_pages.csv saved
page 118 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:28<00:00,  2.68s/it]


tlou2_df_latest_118_pages.csv saved
page 119 get ret


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:39<00:00,  2.80s/it]


tlou2_df_latest_119_pages.csv saved
